# Analisis emisiones co2 Argentina

Vamos a utilizar los dataframes provistos por https://datos.bancomundial.org/.

Debido a falta de informacion, el periodo que se utilizara para el analisis sera de 1990-2018.

<h3>Idea del proyecto</h3>

* Extraer la informacion.
* Limpiar informacion. 
* Juntar los diferentes dataframes en uno en comun.
* Filtraremos y estudiaremos el caso particular de Argentina. 
* Seleccion de variable a predecir.
* Seleccion de features.
* Plantear el modelo de validacion.
* Declaracion de modelo de ML.
* Entrenar el modelo.
* Analizar resultados.



In [77]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
path_co2_em = "/content/gdrive/MyDrive/data_science/emisiones_co2_kt.csv"
path_area_selv = "/content/gdrive/MyDrive/data_science/area_selvatica.csv"
path_cons_elec = "/content/gdrive/MyDrive/data_science/consumo_de_energia_electrica.csv"
path_prod_elec_ren = "/content/gdrive/MyDrive/data_science/produccion_energia_electrica_renovable.csv"


# estos seran los features seleccionados.
# dataframe area selvatica
df_ar_s = pd.read_csv(path_area_selv)

# dataframe co2 emitido
df_co2 = pd.read_csv(path_co2_em)

# dataframe consumo electrico
df_con_elc = pd.read_csv(path_cons_elec)

# dataframe produc electrica renovable
df_prod_elc_ren = pd.read_csv(path_prod_elec_ren)

In [62]:
# eliminamos columnas con valores nulos.

array_columns = np.arange(1960, 1990).astype(str)
df_co2.drop(columns=array_columns, inplace=True)
df_ar_s.drop(columns=array_columns, inplace=True)
df_con_elc.drop(columns=array_columns, inplace=True)
df_prod_elc_ren.drop(columns=array_columns, inplace=True)

In [63]:
# filtramos para obtener los datos sobre Argentina ya que es el que nos interesa.
# tambien dropeamos las columnas con informacion no deseada.

arg_co2_em = df_co2[df_co2["Country Code"] == "ARG"].drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
arg_ar_s = df_ar_s[df_ar_s["Country Name"] == "Argentina"].drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
arg_con_elc = df_con_elc[df_con_elc["Country Name"] == "Argentina"].drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
arg_prd_elc_ren = df_prod_elc_ren[df_prod_elc_ren["Country Name"] == "Argentina"].drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])

In [64]:
# transponemos la matris, ya que es mas practico tener de filas a los anos y en las columnas los features que hemos decidido analizar previamente.

arg_co2_em = arg_co2_em.transpose()
arg_ar_s = arg_ar_s.transpose()
arg_con_elc = arg_con_elc.transpose()
arg_prd_elc_ren = arg_prd_elc_ren.transpose()

In [65]:
# renombramos las columnas

arg_co2_em.columns = ["co2Emission"]
arg_ar_s.columns = ["jungleArea"]
arg_con_elc.columns = ["electrCons"]
arg_prd_elc_ren.columns = ["renElectrProd"]

In [66]:
#creamos el nuevo dataframe que tendra dichas columnas
df_clean = pd.DataFrame()
df_clean["jungleArea"] = arg_ar_s
df_clean["electrCons"] = arg_con_elc
df_clean["renElectrProd"] = arg_prd_elc_ren


In [67]:
# seleccionamos la variable a predecir
Y = arg_co2_em.co2Emission.dropna()

# especificamos los features
test_features = ["jungleArea", "electrCons", "renElectrProd"]
X = df_clean[test_features].dropna()

In [68]:
Y.drop(labels=["2015", "2016", "2017", "2018"], inplace=True)

In [69]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

train_X, val_X, train_Y, val_Y = train_test_split(X, Y, random_state=0, test_size = 0.5)

# definimos el modelo de ML a utilizar
co2_pred_model = DecisionTreeRegressor(random_state=1)

# entrenamos el modelo
co2_pred_model.fit(train_X, train_Y)


DecisionTreeRegressor(random_state=1)

In [70]:
# obtenemos la prediccion de la validacion y comparamos el error.

val_predictions = co2_pred_model.predict(val_X)
print(mean_absolute_error(val_Y, val_predictions))

5222.307692307692


Vemos que tenemos un error aproximado de 5222 kt.

Ahora podemos hacer un analisis mas extenso de este valor obtenido.
Si nos fijamos cual es el promedio de emisiones de co2 de parte de Argentina veremos que:



In [76]:
print(f"Argentina tiene un promedio de emision de co2 de : {np.mean(arg_co2_em)[0]}")

Argentina tiene un promedio de emision de co2 de :145638.27586206896


Por lo que tener un error de 5222 kt parece indicar que nuestro modelo no es muy exacto.

Esto podria deberse a que nuestro modelo pueda estar underfitting o que nuestra seleccion de features no haya sido muy inteligente.

Vamos a buscar mas features para ayudar a nuestro modelo a predecir mejor.